## AHP 가중치 계산

In [1]:
import pandas as pd
df = pd.read_csv('../결합데이터/결합 데이터_1(법정동)/법정동_클러스터링_적용.csv', header=0, encoding='cp949', engine='python')
df.set_index('gid', inplace=True)
전체칼럼 = df.columns.tolist()
print ("칼럼명 확인 후 필요한 칼럼명만 복사 .\n", 전체칼럼)

칼럼명 확인 후 필요한 칼럼명만 복사 .
 ['자치구명', '법정동명', '등록동물수', '교통량', '공시지가음수', '반려동물_관련시설']


In [2]:
점수칼럼 =  ['등록동물수', '교통량', '공시지가음수', '반려동물_관련시설']
print("입력한 점수컬럼명은 다음과 같음\n", 점수칼럼)

가중치 = [1] * len(점수칼럼)

입력한 점수컬럼명은 다음과 같음
 ['등록동물수', '교통량', '공시지가음수', '반려동물_관련시설']


In [3]:
df

,자치구명,법정동명,등록동물수,교통량,공시지가음수,반려동물_관련시설
gid,,,,,,
11170101,용산구,후암동,1223,-9225,-3663212,3
11170102,용산구,용산동2가,1021,-8941,-3129218,3
11170103,용산구,용산동4가,2,-8941,-3207603,4
11170104,용산구,갈월동,187,-3741,-8978836,0
11170105,용산구,남영동,35,-3741,-11143536,0
...,...,...,...,...,...,...
11740106,강동구,둔촌동,1524,-9257,-2366818,8
11740107,강동구,암사동,3083,-6205,-1849343,12
11740108,강동구,성내동,2971,-14826,-5293907,21


### AHP 가중치 선언 
- 각 변수에 대한 가중치를 선언

In [4]:
가중치 = [0.1610, 0.4562,0.1706, 0.2122]      # 점수칼럼 개수 만큼, 구분은 쉼표

if (len(가중치) != len(점수칼럼)):
    print("입력한 가중치 개수가 점수칼럼 개수와 다르므로 확인 후 다시 입력 바랍니다")
else:
    print("다음 줄을 실행 하세요")

다음 줄을 실행 하세요


In [7]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# 점수칼럼 정규화
norm = df.copy()

norm[점수칼럼] = scaler.fit_transform(norm.loc[:,점수칼럼])
norm.reset_index(inplace=True)
################## 정규화 끝################################################################################


# 총점 계산
df_norm = norm.copy()

for i in range(0, len(점수칼럼)):
    name = 점수칼럼[i]
    df_norm[name] = df_norm[name] * 가중치[i] 

df_norm.set_index(['gid','자치구명', '법정동명'], inplace=True)

df_norm['sum'] = df_norm.sum(1)
df_norm.reset_index(inplace=True)

# 등수 계산 
df_norm['순위'] = df_norm['sum'].rank(method='dense', ascending=False).astype(int)

final = df_norm.sort_values(by='순위')
final.reset_index(drop=True, inplace=True)

# 상위 10개만 학인
display("결과", final.head(10))

# 최종 결과 CSV 파일쓰기
final.to_csv('결과_동단위.csv', index=False, sep=',', encoding='cp949')

'결과'

,gid,자치구명,법정동명,등록동물수,교통량,공시지가음수,반려동물_관련시설,sum,순위
0,11500103,강서구,화곡동,0.139010,0.427547,0.134581,0.196673,0.897810,1
1,11620102,관악구,신림동,0.161000,0.377342,0.152386,0.191498,0.882226,2
2,11470101,양천구,신정동,0.081346,0.363369,0.127493,0.212200,0.784407,3
3,11350105,노원구,상계동,0.121179,0.308254,0.157032,0.191498,0.777962,4
4,11260101,중랑구,면목동,0.084322,0.386152,0.148880,0.144917,0.764272,5
5,11560132,영등포구,신길동,0.055772,0.421856,0.124440,0.098337,0.700405,6
6,11620101,관악구,봉천동,0.106201,0.336591,0.136916,0.103512,0.683220,7
7,11380104,은평구,갈현동,0.037764,0.438717,0.141647,0.062107,0.680236,8
8,11380107,은평구,응암동,0.052686,0.404183,0.133386,0.087985,0.678240,9
9,11650101,서초구,방배동,0.059279,0.355823,0.104532,0.139741,0.659375,10
